In [6]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import (SimpleImputer, KNNImputer, IterativeImputer)

train_features = pd.read_csv('train_features.csv', index_col=0)
train_labels   = pd.read_csv('train_labels.csv', index_col=0)
test_features  = pd.read_csv('test_features.csv', index_col=0)

x_train = train_features.values[:,1:]
y_train = train_labels.values
x_test = test_features.values[:,1:]

Id_pat = train_features.index
Id_lab = train_labels.index
Id_pat_test = test_features.index

In [7]:
train_mean = np.nanmean(x_train, axis = 0)
test_mean = np.nanmean(x_test, axis = 0)

unique_ids_train = pd.unique(Id_pat)
unique_lab_ids_train = pd.unique(Id_lab)
unique_ids_test = pd.unique(Id_pat_test)

imputer = SimpleImputer(missing_values=np.nan, strategy="median", fill_value=None)
#print(pd.DataFrame(imputer.fit_transform(x_train)).to_csv("tmp.csv"))


None


In [8]:
# helper functions

def pat_data_arr(index):
    return x_train[index*12:index*12+12, :]

def pat_data_test_arr(index):
    return x_test[index*12:index*12+12, :]

In [9]:
x_imp = np.zeros((int(x_train.shape[0]/12), 35))

for pid in range(len(unique_ids_train)):
    # get training data for a patient (dim 12 x d)
    x_patient = pat_data_arr(pid)
    x_patient_imp = np.array([0.]*35)
    
    # deal with column seperately
    for col in range(len(x_patient[1, :])):
        # Count number of Nan in each column
        nan_count = np.isnan(x_patient[:, col]).sum()
        
        if nan_count<12:
            x_patient_imp[col]=np.nanmean(x_patient, axis = 0)[col]
        else:
            x_patient_imp[col] = -1           
    # Concatenate imputed data for every patient
    x_imp[pid, :] = x_patient_imp
    
print(pd.DataFrame(x_imp).to_csv("tmp.csv"))

<ipython-input-9-f8ebf3808628>:14: RuntimeWarning: Mean of empty slice
  x_patient_imp[col]=np.nanmean(x_patient, axis = 0)[col]
[[ 3.40000000e+01 -1.00000000e+00 -1.00000000e+00 ... -1.00000000e+00
   1.14500000e+02  7.37000000e+00]
 [ 7.10000000e+01 -1.00000000e+00  2.78000000e+01 ...  1.00000000e-02
   1.32909091e+02 -1.00000000e+00]
 [ 6.80000000e+01 -1.00000000e+00  2.09000000e+01 ... -1.00000000e+00
   1.17000000e+02 -1.00000000e+00]
 ...
 [ 5.30000000e+01 -1.00000000e+00 -1.00000000e+00 ... -1.00000000e+00
   9.92500000e+01  7.40250000e+00]
 [ 8.90000000e+01 -1.00000000e+00  3.47000000e+01 ...  1.50500000e+00
   1.31545455e+02 -1.00000000e+00]
 [ 8.50000000e+01 -1.00000000e+00  3.64000000e+01 ... -1.00000000e+00
   1.28090909e+02 -1.00000000e+00]]
